# 工厂函数（Factory Function）
- 外层函数（工厂函数）定义了一些变量。
- 内层函数（返回的函数）捕获这些变量的引用。
- 返回的内层函数形成闭包，保留对外层函数变量的访问能力。
- 如果内层函数需要修改外层函数的变量，必须使用 **nonlocal** 声明，否则 Python 会认为是在创建局部变量。
- 虽然工厂函数通常利用闭包机制，但并非所有工厂函数都是闭包。如果工厂函数返回的内层函数不依赖外层函数的变量（即没有捕获自由变量），则不构成闭包。

In [1]:
def make_adder(x):
    def adder(y):
        return x + y

    return adder


adder100 = make_adder(100)
adder200 = make_adder(200)
print(adder100(10))  # 输出: 110
print(adder200(10))  # 输出: 210

110
210


In [1]:
def make_counter():
    count = 0

    def counter():
        nonlocal count  # 使用 nonlocal 关键字来修改外层函数的变量
        count += 1
        return count

    return counter


counter1 = make_counter()
counter2 = make_counter()

counter1()
counter1()
counter1()

counter2()
counter2()
counter2()
counter2()

print(counter1())
print(counter2())

4
5


In [2]:
# 外层变量的修改会影响闭包的行为
def make_funcs():
    funcs = []
    for i in range(3):
        def func():
            return i  # 注意这里的 i 是外层变量

        funcs.append(func)
    return funcs


three_funcs = make_funcs()
print(three_funcs[0]())  # 输出: 2
print(three_funcs[1]())  # 输出: 2
print(three_funcs[2]())  # 输出: 2

2
2
2


In [3]:
def make_funcs_fixed():
    funcs = []
    for i in range(3):
        def func(x=i):  # 使用默认参数来捕获当前的 i 值
            return x

        funcs.append(func)
    return funcs


three_funcs_fixed = make_funcs_fixed()
print(three_funcs_fixed[0]())  # 输出: 0
print(three_funcs_fixed[1]())  # 输出: 1
print(three_funcs_fixed[2]())  # 输出: 2

0
1
2


# 闭包（Closure）
函数在定义时就确定了其作用域，而不是在调用时。
- 在异步编程中，闭包常用于回调函数以保留上下文。
- 函数工厂，动态生成具有特定行为的函数。
- 变量封装，闭包可以隐藏变量，防止外部直接访问，从而实现数据私有化。

In [5]:
# 自定义学习率调度器
import torch
from torch import nn, optim


def make_lr_scheduler(initial_lr, decay_rate):
    def lr_scheduler(step):
        lr = initial_lr / (1 + decay_rate * step)
        return lr

    return lr_scheduler

LR_0 = 1e-3
lr_scheduler = make_lr_scheduler(initial_lr=LR_0, decay_rate=0.1)
model = nn.Sequential(nn.Linear(10, 20), nn.ReLU(), nn.Linear(20, 1))
optimizer = optim.Adam(model.parameters(), lr=LR_0)
criterion = nn.MSELoss()

for epoch in range(5):
    inputs = torch.randn(32, 10)  # 假设输入数据
    targets = torch.randn(32, 1)  # 假设目标数据

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    # 更新学习率
    curr_lr = lr_scheduler(epoch)
    for param_group in optimizer.param_groups:
        param_group['lr'] = curr_lr
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}, Learning Rate: {curr_lr:.4f}')


Epoch 1, Loss: 1.4223, Learning Rate: 0.0010
Epoch 2, Loss: 0.7373, Learning Rate: 0.0009
Epoch 3, Loss: 1.1607, Learning Rate: 0.0008
Epoch 4, Loss: 0.8987, Learning Rate: 0.0008
Epoch 5, Loss: 0.9922, Learning Rate: 0.0007


In [7]:
# 自定义优化器
import torch
from torch import nn, optim

def make_loss_closure(model, criterion, inputs, targets):
    def closure():
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        return loss
    return closure

model = nn.Sequential(nn.Linear(10, 20), nn.ReLU(), nn.Linear(20, 1))
optimizer = optim.LBFGS(model.parameters(), lr=0.1) # LBFGS 优化器需要闭包
criterion = nn.MSELoss()
inputs = torch.randn(32, 10)  # 假设输入数据
targets = torch.randn(32, 1)  # 假设目标数据
loss_closure = make_loss_closure(model, criterion, inputs, targets)
for epoch in range(5):
    optimizer.step(loss_closure)  # 调用闭包计算损失并更新参数
    print(f'Epoch {epoch+1}, Loss: {loss_closure().item():.6f}')

Epoch 1, Loss: 0.166608
Epoch 2, Loss: 0.006464
Epoch 3, Loss: 0.000283
Epoch 4, Loss: 0.000006
Epoch 5, Loss: 0.000000


In [8]:
# 回调函数
import torch
from torch import nn, optim

def make_train_callback():
    metrics = {"losses": [], "epoch": 0}
    def callback(model, loss):
        metrics["losses"].append(loss.item())
        metrics["epoch"] += 1
        print(f'Epoch {metrics["epoch"]}, Loss: {loss.item():.4f}')
        if metrics['epoch'] % 2 == 0:
            torch.save(model.state_dict(), f'model_epoch_{metrics["epoch"]}.pth')
    return callback

model = nn.Sequential(nn.Linear(10, 20), nn.ReLU(), nn.Linear(20, 1))
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.MSELoss()
callback = make_train_callback()

for epoch in range(5):
    inputs = torch.randn(32, 10)
    targets = torch.randn(32, 1)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    callback(model, loss)  # 调用回调函数

Epoch 1, Loss: 0.8360
Epoch 2, Loss: 0.8866
Epoch 3, Loss: 1.0002
Epoch 4, Loss: 1.1965
Epoch 5, Loss: 1.1751
